In [7]:
import torchvision.models as models
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
from torch.optim import Adam
from tkr_generator import *
import matplotlib.pyplot as plt
import random
import os
from PIL import Image
import numpy as np
import cv2

In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np

class channel1(nn.Module):
	def __init__(self):
		super(channel1, self).__init__()
		self.conv1 = nn.Conv2d(1, 96, kernel_size=3, stride=1, padding=1)
		self.relu1 = nn.ReLU(inplace=True)
		self.conv2 = nn.Conv2d(96, 10, kernel_size=3, stride=1, padding=1)
		self.conv3 = nn.Conv2d(10, 4, kernel_size=3, stride=1, padding=1)
		self.fc1 = nn.Linear(16 * 5 * 5, 120)
		self.pool = nn.MaxPool2d(2, 2)
		self.fc1 = nn.Linear(4096, 120)
		self.fc2 = nn.Linear(120, 1)

	def forward(self, x):
		x = self.conv1(x)
		x = self.pool(self.relu1(x))
		
		x = self.conv2(x)
		x = self.pool(self.relu1(x))

		x = self.conv3(x)
		x = self.pool(self.relu1(x))


		x = torch.flatten(x, 1)
		x = self.fc1(x)
		x = self.fc2(x)

		return x

In [8]:
class CDDataset(torch.utils.data.Dataset):
    def __init__(self):
        self.cat_fn = os.listdir('../cat_dog/cat')
        self.dog_fn = os.listdir('../cat_dog/dog')
        super().__init__()
    
    def __getitem__(self, idx):
        img_lst = []
        target_lst = []
        for i in range(4):
            if random.random() > 0.5:
                sample = random.choice(self.cat_fn)
                target = 0
                im = np.array(Image.open(os.path.join('../cat_dog/cat', sample)))[:,:,1].astype(np.float32)

                
            else:
                sample = random.choice(self.dog_fn)
                target = 1
                im = np.array(Image.open(os.path.join('../cat_dog/dog', sample)))[:,:,1].astype(np.float32)
            
            img_s=cv2.resize(im, (256, 256))
            img_s = cv2.normalize(img_s, None, 0, 1, cv2.NORM_MINMAX)
            img_lst.append(img_s)
            target_lst.append(target)
            imgs = np.stack(img_lst, axis=0)
            targets = np.stack(target_lst, axis=0)
        return imgs, targets


In [21]:
# Param = {
#     'LR': 0.0001,
#     'WC': 0.0001
# }
# num_epochs = 1
# model_type = 'densenet'
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# print("The model will be running on", device, "device")
# # Convert model parameters and buffers to CPU or Cuda
# model = channel1().float()
# loss_fn = nn.BCELoss()
# optimizer = Adam(model.parameters(), lr=Param['LR'], weight_decay=Param['WC'])
# best_accuracy = 0.0

# print(os.getcwd())
# train_loader = CDDataset()


# # Define your execution device

for epoch in range(num_epochs):  # loop over the dataset multiple times
    running_loss = 0.0
    running_acc = 0.0
    for i, (images, labels) in enumerate(train_loader, 0):
        # get the inputs
        images = torch.tensor(images, dtype=float).to(device, dtype=float)
        labels = torch.tensor(labels, dtype=float).to(device)

        # zero the parameter gradients
        optimizer.zero_grad()
        # predict classes using images from the training set
        outputs = model(images.reshape(4,1,256,256).float())
        pred = torch.sigmoid(outputs)
        
        # compute the loss based on model output and real labels
        loss = loss_fn(pred[:,0], labels.float())
        # backpropagate the loss
        loss.backward()
        # adjust parameters based on the calculated gradients
        optimizer.step()

        # Let's print statistics for every 1,000 images
        running_loss += loss.item()     # extract the loss value
        if i % 20 == 0:    
            # print every 1000 (twice per epoch) 
            print('[%d, %5d] loss: %.3f' %
                    (epoch + 1, i + 1, running_loss / 20))
            # zero the loss
            if running_loss / 20 < 0.3:
                 torch.save(model.state_dict(), 'weights.pth')
            running_loss = 0.0

[1,     1] loss: 0.015
[1,    21] loss: 0.543
[1,    41] loss: 0.588
[1,    61] loss: 0.503
[1,    81] loss: 0.491
[1,   101] loss: 0.548
[1,   121] loss: 0.600
[1,   141] loss: 0.500
[1,   161] loss: 0.555
[1,   181] loss: 0.549
[1,   201] loss: 0.496
[1,   221] loss: 0.505
[1,   241] loss: 0.502
[1,   261] loss: 0.593
[1,   281] loss: 0.561
[1,   301] loss: 0.470
[1,   321] loss: 0.579
[1,   341] loss: 0.636
[1,   361] loss: 0.576
[1,   381] loss: 0.471
[1,   401] loss: 0.489
[1,   421] loss: 0.599
[1,   441] loss: 0.594
[1,   461] loss: 0.492
[1,   481] loss: 0.577
[1,   501] loss: 0.554
[1,   521] loss: 0.480
[1,   541] loss: 0.530
[1,   561] loss: 0.523
[1,   581] loss: 0.516
[1,   601] loss: 0.544
[1,   621] loss: 0.514
[1,   641] loss: 0.523
[1,   661] loss: 0.554
[1,   681] loss: 0.562
[1,   701] loss: 0.549
[1,   721] loss: 0.535
[1,   741] loss: 0.575
[1,   761] loss: 0.529
[1,   781] loss: 0.502
[1,   801] loss: 0.527
[1,   821] loss: 0.394
[1,   841] loss: 0.576
[1,   861] 

In [10]:
model

channel1(
  (conv1): Conv2d(1, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu1): ReLU(inplace=True)
  (conv2): Conv2d(96, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(10, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=4096, out_features=120, bias=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc2): Linear(in_features=120, out_features=1, bias=True)
)

In [20]:
count = 0
for i in range(100):
    images, labels = train_loader.__getitem__(0)
    images = torch.tensor(images, dtype=float)
    outputs = model(images.reshape(4,1,256,256).float())
    pred = torch.round(torch.sigmoid(outputs)).detach().numpy().squeeze()
    count += np.sum(pred == labels)
count

287

In [18]:
285/400

0.7125

In [13]:
model2 = channel1().float()

In [14]:
count = 0
for i in range(100):
    images, labels = train_loader.__getitem__(0)
    images = torch.tensor(images, dtype=float)
    outputs = model2(images.reshape(4,1,256,256).float())
    pred = torch.round(torch.sigmoid(outputs)).detach().numpy().squeeze()
    count += np.sum(pred == labels)
count

198

In [15]:
280/400

0.7